In [ ]:
# ------------------------------
# Step 1. Import Libraries
# ------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------
# Step 2. Load and Clean Data
# ------------------------------
file_path = "dataset.xlsx"   # <-- change path if needed
df = pd.read_excel(file_path, sheet_name="Data")



# Replace -1 with NaN only in numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].replace(-1, np.nan)

# ------------------------------
# Step 3. Approximate Ultimate Analysis
# ------------------------------
# Reference biochemical elemental composition (wt%)
ref = {
    "Cellulose": {"C":44.4,"H":6.2,"O":49.4,"N":0},
    "HemiCellulose": {"C":44.4,"H":6.2,"O":49.4,"N":0},
    "Sugar": {"C":40.0,"H":6.7,"O":53.3,"N":0},
    "Lignin": {"C":63.4,"H":6.0,"O":30.6,"N":0},
    "Proteins": {"C":53.0,"H":7.0,"O":22.0,"N":18.0},
    "Lipids": {"C":77.0,"H":12.0,"O":11.0,"N":0},
}

# Initialize new columns
for elem in ["C","H","O","N"]:
    df[f"{elem}_approx"] = 0.0

# Accumulate contributions row by row
for comp, formula in ref.items():
    if comp in df.columns:
        frac = (df[comp].astype(float).fillna(0.0))/100.0
        for elem in ["C","H","O","N"]:
            df[f"{elem}_approx"] += frac * formula[elem]

# Coverage check (how much biomass is described by biochemical fractions)
df["approx_coverage_frac"] = df[[c for c in ref.keys() if c in df.columns]].fillna(0).sum(axis=1)/100

# Normalize so C+H+O+N = 100 (only where row sum > 0)
row_sums = df[["C_approx","H_approx","O_approx","N_approx"]].sum(axis=1)
mask = row_sums > 0
df.loc[mask, ["C_approx","H_approx","O_approx","N_approx"]] = (
    df.loc[mask, ["C_approx","H_approx","O_approx","N_approx"]]
      .div(row_sums[mask], axis=0)*100
)

# Flag low coverage rows (<60%)
df["low_coverage_flag"] = df["approx_coverage_frac"] < 0.6

# ------------------------------
# Step 4. Add Beeswax Row
# ------------------------------
beeswax = {
    "Author": "Literature",
    "Ref": "Beeswax_lit",
    "Resource": "Beeswax",
    "Details": "Unpurified Beeswax (literature)",
    "C_approx": 82.0,
    "H_approx": 14.0,
    "O_approx": 3.0,
    "N_approx": 0.5,
    "HHVResource": 38.0   # placeholder, update if you have experimental HHV
}

beeswax_df = pd.DataFrame([beeswax])

# Calculate ratios for beeswax
beeswax_df["H_C"] = (beeswax_df["H_approx"]/1.0) / (beeswax_df["C_approx"]/12.0)
beeswax_df["O_C"] = (beeswax_df["O_approx"]/16.0) / (beeswax_df["C_approx"]/12.0)
beeswax_df["N_C"] = (beeswax_df["N_approx"]/14.0) / (beeswax_df["C_approx"]/12.0)

# Append beeswax row to dataset
df_all = pd.concat([df, beeswax_df], ignore_index=True)

# ------------------------------
# Step 5. Calculate Ratios (for all rows)
# ------------------------------
df_all["H_C"] = np.where(df_all["C_approx"]>0, (df_all["H_approx"]/1.0)/(df_all["C_approx"]/12.0), np.nan)
df_all["O_C"] = np.where(df_all["C_approx"]>0, (df_all["O_approx"]/16.0)/(df_all["C_approx"]/12.0), np.nan)
df_all["N_C"] = np.where(df_all["C_approx"]>0, (df_all["N_approx"]/14.0)/(df_all["C_approx"]/12.0), np.nan)

# ------------------------------
# Step 6. Plot Van Krevelen Panels (with Cluster Annotations)
# ------------------------------
vk = df_all.dropna(subset=["H_C","O_C","N_C"])
bw_mask = vk["Resource"].str.contains("Beeswax", case=False, na=False)

fig, axes = plt.subplots(1, 2, figsize=(15,6))

# --- Panel A: H/C vs O/C ---
sc1 = axes[0].scatter(vk["O_C"], vk["H_C"],
                      c=vk["HHVResource"], cmap="viridis",
                      s=60, edgecolor="k", alpha=0.8,
                      label="Literature Biomass")

axes[0].scatter(vk.loc[bw_mask,"O_C"], vk.loc[bw_mask,"H_C"],
                c="red", marker="*", s=250, edgecolor="k",
                label="Unpurified Beeswax")

# Subfigure label
axes[0].text(0.02,0.85,"(A)", transform=axes[0].transAxes,
             fontsize=14, fontweight="bold")

# Axis labels
axes[0].set_xlabel("O/C molar ratio", fontsize=12)
axes[0].set_ylabel("H/C molar ratio", fontsize=12)

# Reference lines
axes[0].axhline(y=1.5, color="gray", linestyle="--", lw=1)
axes[0].axvline(x=0.5, color="gray", linestyle="--", lw=1)

# Cluster annotations with arrows
axes[0].annotate("Carbohydrate-rich", xy=(0.8,1.7), xytext=(1.2,2.2),
                 arrowprops=dict(facecolor="brown", arrowstyle="->"),
                 fontsize=10, color="brown")

axes[0].annotate("Lignin-rich", xy=(0.3,1.0), xytext=(0.6,0.6),
                 arrowprops=dict(facecolor="darkgreen", arrowstyle="->"),
                 fontsize=10, color="darkgreen")

axes[0].annotate("Lipid-rich\n(Oils, Beeswax)", xy=(0.05,2.0), xytext=(0.3,2.3),
                 arrowprops=dict(facecolor="darkblue", arrowstyle="->"),
                 fontsize=10, color="darkblue")

axes[0].annotate("Protein-rich", xy=(0.2,1.5), xytext=(0.05,1.1),
                 arrowprops=dict(facecolor="purple", arrowstyle="->"),
                 fontsize=10, color="purple")

# Colorbar
cbar = fig.colorbar(sc1, ax=axes[0])
cbar.set_label("HHV (MJ/kg)", fontsize=12)

# Legend
axes[0].legend(frameon=True, fontsize=10, loc="upper right")


# --- Panel B: N/C vs O/C ---
sc2 = axes[1].scatter(vk["O_C"], vk["N_C"],
                      c=vk["HHVResource"], cmap="viridis",
                      s=60, edgecolor="k", alpha=0.8,
                      label="Literature Biomass")

axes[1].scatter(vk.loc[bw_mask,"O_C"], vk.loc[bw_mask,"N_C"],
                c="red", marker="*", s=250, edgecolor="k",
                label="Unpurified Beeswax")

# Subfigure label
axes[1].text(0.02,0.85,"(B)", transform=axes[1].transAxes,
             fontsize=14, fontweight="bold")

# Axis labels
axes[1].set_xlabel("O/C molar ratio", fontsize=12)
axes[1].set_ylabel("N/C molar ratio", fontsize=12)

# Cluster annotations for N/C space
axes[1].annotate("Protein-rich / N-rich", xy=(0.3,0.15), xytext=(0.6,0.25),
                 arrowprops=dict(facecolor="purple", arrowstyle="->"),
                 fontsize=10, color="purple")

axes[1].annotate("Low-N biomass\n(Carbohydrates, Lipids)", xy=(0.05,0.02), xytext=(0.7,0.2),
                 arrowprops=dict(facecolor="blue", arrowstyle="->"),
                 fontsize=10, color="blue")

# Colorbar
cbar2 = fig.colorbar(sc2, ax=axes[1])
cbar2.set_label("HHV (MJ/kg)", fontsize=12)

# Legend
axes[1].legend(frameon=True, fontsize=10, loc="upper right")


# ------------------------------
# Save high-resolution versions
# ------------------------------
plt.tight_layout()
plt.savefig("van_krevelen_with_clusters.png", dpi=300, bbox_inches="tight")
plt.savefig("van_krevelen_with_clusters.tiff", dpi=300, bbox_inches="tight")
plt.savefig("van_krevelen_with_clusters.pdf", bbox_inches="tight")
plt.show()


#show first 5 row
#df.head()

# see all column name
print(df.columns)

# Check general info
df.info()

# Get summary statistics for numeric columns
df.describe()

